In [1]:
import pickle 
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import punkt
from nltk.corpus.reader import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
#Trained models
root_path = r'D:\Learn\research\Text_Classification2\data'
path_models = root_path+'\\Models'
#GBC
path_gbc = path_models + '\\best_gbc.pickle'

print(path_gbc)
with open(path_gbc,'rb') as data:
    gbc_model = pickle.load(data)
    

D:\Learn\research\Text_Classification2\data\Models\best_gbc.pickle


In [3]:
#TF-IDF Object
path_tfidf = root_path+'\\Pickles\\tfidf.pickle'
with open(path_tfidf, 'rb') as data:
    tfidf = pickle.load(data)

In [4]:
#Category mapping dictionary

category_codes = {
    'business': 0,
    'entertainment': 1,
    'politics': 2,
    'sport': 3,
    'tech': 4,
    'ncovid':5,
}

In [5]:
#Future engineering workflow

punctuation_signs = list("?:!.,;")
stop_words = list(stopwords.words('english'))

def create_features_from_text(text):
    
    # Dataframe creation
    lemmatized_text_list = []
    df = pd.DataFrame(columns=['Content'])
    df.loc[0] = text
    df['Content_Parsed_1'] = df['Content'].str.replace("\r", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")
    df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')
    df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()
    df['Content_Parsed_3'] = df['Content_Parsed_2']
    for punct_sign in punctuation_signs:
        df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')
    df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_list = []
    text = df.loc[0]['Content_Parsed_4']
    text_words = text.split(" ")
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
    lemmatized_text = " ".join(lemmatized_list)    
    lemmatized_text_list.append(lemmatized_text)
    df['Content_Parsed_5'] = lemmatized_text_list
    df['Content_Parsed_6'] = df['Content_Parsed_5']
    for stop_word in stop_words:
        regex_stopword = r"\b" + stop_word + r"\b"
        df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')
    
    df_main = df['Content_Parsed_6']
    #df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})
    
    # TF-IDF
    features = tfidf.transform(df_main).toarray()
    
    return features
    

In [6]:
#Function that tells us the category given the category code:
def get_category_name(category_id):
    for category, id_ in category_codes.items():    
        if id_ == category_id:
            return category

In [7]:
def predict_from_text(text):
    
    
    # Predict using the input model
    prediction_gbc = gbc_model.predict(create_features_from_text(text))[0]
    prediction_gbc_proba = gbc_model.predict_proba(create_features_from_text(text))[0]
    
    # Return result
    category_gbc = get_category_name(prediction_gbc)
    
    print("The predicted category using the GBC model is %s." %(category_gbc) )
    print("The conditional probability is: %a" %(prediction_gbc_proba.max()*100))

In [8]:
text = """

The center-right party Ciudadanos closed a deal on Wednesday with the support of the conservative Popular Party (PP) to take control of the speaker’s committee in the Andalusian parliament, paving the way for the regional PP leader, Juan Manuel Moreno, to stand as the candidate for premier of the southern Spanish region. The move would see the Socialist Party (PSOE) lose power in the Junta, as the regional government is known, for the first time in 36 years.

Talks in Andalusia have been ongoing since regional polls were held on December 2. The PSOE, led by incumbent premier Susana Díaz, had been expected to win the early elections, but in a shock result the party took the most seats in parliament, 33, but fell well short of a majority of 55. It was their worst result in the region since Spain returned to democracy. The PP came in second, with 26 seats, while Ciudadanos were third with 21. The major surprise was the strong performance of far-right group Vox, which won more than 391,000 votes (10.9%), giving it 12 deputies. The anti-immigration group is the first of its kind to win seats in a Spanish parliament since the end of the Francisco Franco dictatorship. It now holds the key to power in Andalusia, given that its votes, added to those of the PP and Ciudadanos, constitute an absolute majority.

The move would see the Socialist Party lose power in the region for the first time in 36 years

On Thursday, Marta Bosquet of Ciudadanos was voted in as the new speaker of the Andalusian parliament thanks to 59 votes from her party, the PP and Vox. The other candidate, Inmaculada Nieto of Adelante Andalucía, secured 50 votes – from her own party and 33 from the PSOE.

The speaker’s role in the parliament is key for the calling of an investiture vote and for the selection of the candidate for premier.

Officially, the talks as to the make up of a future government have yet to start, but in reality they are well advanced, according to sources from both the PP and Ciudadanos. The leader of the Andalusian PP is banking on being voted into power around January 16 and wants the majority of his Cabinet to be decided “five days before the investiture vote.”

The speaker’s role in the parliament is key for the calling of an investiture vote and for the selection of the candidate for premier

The PP, which was ousted from power by the PSOE in the national government in June, is keen to take the reins of power in Andalusia as soon as possible. The difficulties that Ciudadanos has faced to justify the necessary inclusion of Vox in the talks, has slowed down progress. Rather than align itself with the far right party, the group – which began life in Catalonia in response to the independence drive, but soon launched onto the national stage – had sought a deal with Adelante Andalucía.

Wednesday was a day of intense talks among the parties in a bid to find a solution that would keep everyone happy. But at 9pm last night, Adelante Andalucía announced that it would not be part of “any deal” and that would instead vote for its own candidates to the speaker’s committee in order to “face up to the right wing and the extreme right.”

The PSOE, meanwhile, argues that having won the elections with a seven-seat lead over the PP gives it the legitimacy to aspire to the control of the regional government and the parliament, and to maintain its positions on the speaker’s committee.



"""

In [9]:
predict_from_text(text)

The predicted category using the GBC model is politics.
The conditional probability is: 97.20487224607936


In [10]:
# Politics

text = """Disputes have already broken out within the new political alliance that is working to end 36 years of rule by the Socialist Party (PSOE) in Spain’s southern region of Andalusia.

Just hours after the far-right Vox agreed to support the Popular Party (PP)’s candidate to head the region, Juan Manuel Moreno, its demand for a specific regional department for family affairs is running into opposition from Ciudadanos (Citizens), the third party involved in the government change.

These early clashes suggest it could be difficult to export the model to other parts of Spain

The PP and the liberal Ciudadanos have reached their own governing agreement in the wake of an election that undermined the Socialists’ hold on power, but they need support from Vox’s 12 newly elected lawmakers to make it a reality.

Ciudadanos has refused point-blank to meet with Vox representatives, but the PP has struck its own parallel deal with the far right to ensure its support at the investiture debate, which will take place on January 15 and 16.

On Friday morning, Juan Marín of Ciudadanos said that there are no plans for a separate family affairs department within the government structure they have devised with their PP partners, and which does not include Vox.

The reform party has insisted that the Vox-PP deal does not affect them at all, and Ciudadanos’ national leader, Albert Rivera, said on Thursday that Vox “has had to take a step back, correct itself, and sheathe its sword,” alluding to the fact that Vox has dropped some of its most radical demands such as the deportation of 52,000 undocumented migrants.

 Vox national leader Santiago Abascal (c) and Andalusian leader Francisco Serrano (r).
Vox national leader Santiago Abascal (c) and Andalusian leader Francisco Serrano (r). REUTERS
But Vox insists on a family department, and said it will expect loyalty from the PP on this issue. Meanwhile, Ciudadanos has also warned it will not support the PP-Vox proposal to change the historical memory law for a “concord law.”

These early clashes suggest it could be difficult to export the model to other parts of Spain, where local and regional elections are scheduled to be held this year. The PP president, Pablo Casado, said on Thursday that their separate deals with Ciudadanos and Vox in Andalusia are “a preamble of what’s going to happen in May in Spain.”

The PP is anxious to win back power in regions like Valencia, the Balearic Islands, Castilla-La Mancha, Aragón and Cantabria, and to retain it in Madrid, Murcia, La Rioja and Castilla y León.

Parliamentary debate
The PSOE has already digested the fact that it is losing its hold on Spain’s most populated region. “We will conduct a responsible opposition, and remain vigilant about defending self-government and Andalusia’s presence in Spain,” said the party’s parliamentary spokesman, Mario Jiménez.

The Socialists will not be putting forward a candidate, now that the PP nominee has enough support for an absolute majority in the first round. The debates will take place on January 15 and 16, followed by a vote, said Speaker Marta Bosquet.

The sum of the PP, Ciudadanos and Vox votes is four above the 55 required for a majority. The PSOE lost 14 seats at the December 2 election, down to 33. The Podemos-backed Adelante Andalucía has 17 lawmakers.

"""

predict_from_text(text)

The predicted category using the GBC model is politics.
The conditional probability is: 98.93362151948484


In [11]:
text = """
Despite football being halted across the globe by the coronavirus pandemic with all major leagues and competitions suspended, the Bundesliga is now eager to return to action with plans to play spectator-free games in May.

Germany has led the way in its approach to combating the coronavirus pandemic. The country's intense programme of consistent testing from early on has resulted in far fewer deaths than the worst-affected countries.

The German Football League (DFL), responsible for operating the Bundesliga and the second division, is determined to finish the current season. Its 36 professional clubs met on Thursday to discuss a restart of the season next month.

However, Germany's football authorities still face some huge questions before they get the green light to resume football in the country behind closed doors.

"If we start on 9 May, we are ready. If it is later, we will be ready again," DFL chief executive Christian Seifert said on Thursday.

"For us, what is decisive is what the politicians will decide. It is not for us to decide when.

"Games without spectators are not what we want - but at the moment the only thing that seems feasible."

So, what challenges do they face and how are they planning to combat them?
"""
predict_from_text(text)

The predicted category using the GBC model is sport.
The conditional probability is: 98.39452560275139


In [12]:
text = """
What will Germany's government say?
German politics has recently decided that large events with crowds will be banned at least until 24 October. That rules out the Bundesliga being played in front of spectators until the scheduled start of next season.

The only way to work around regulations is to stage spectator-free games, with only players, coaching staff, medics, referees, ball boys and ball girls, ground staff, technicians, safety officers, and production staff for television and VAR being present inside the stadiums. League officials estimate that up to 300 people are needed per match.

In Germany, the 16 federal states determine the maximum number of people allowed for public gatherings and a decision about that number will be made by the end of April.

The governments could put a stop to any plans from the DFL, but they are aware of the economic struggles of professional football and are expected to grant the wish of the clubs.

"A weekend with football is much more bearable than a weekend without football. That's why I could picture to have matches behind closed doors", Markus Soder, state premier of Bavaria, said at a recent online talk show hosted by German newspaper Bild.

He and Armin Laschet, state premier of North Rhine-Westphalia, have said they think it's conceivable the Bundesliga would restart as soon as 9 May.

Soder and Laschet rushed ahead with their statements, knowing the federal government and the 16 state governments meet on 30 April, with the likely outcome that football matches are allowed to be staged in the following months.


"""

predict_from_text(text)

The predicted category using the GBC model is sport.
The conditional probability is: 97.09765835333155


In [13]:
text = """
Staging professional football games even with only a few hundred people present becomes a logistical challenge. A few weeks ago, the DFL set up a task force to define the necessary medical and hygiene protocols which would be required.

According to plans of the task force, the stadiums will be divided into three zones. Only a maximum of 100 people can be in each of these sections. The inner section belongs to the 22 players, 18 bench players, five referees and around 53 others. The stands are considered the second section, and the area directly surrounding the stadium is the third one.

The plans also include regular testing of players and coaches between training sessions and before every match. Alexander Kekule, a biochemist from Martin Luther University in Halle, estimates that the DFL would need up to 20,000 detection tests and exceptional organisational measures.

He said: "Players have to be shielded under specific safety rules to prevent infections, because they are not able to avoid contact on the pitch."

Regular testing could create a headache from a practical standpoint, because it is impossible to know whether a player is already infected when he takes to the pitch.

"It takes a couple of days before someone, who has contracted the virus, tests positive," Martin Eichner, an epidemiologist from University of Tubingen, explains. He dismisses the option of faster detection tests that have been in development, as they are not fully reliable yet.

Others have expressed concerns about the moral implications. "Tens of thousands of tests would be needed that are missing in nursing facilities", Karl Lauterbach, health expert for the Social Democratic Party (SPD), recently tweeted. 

The DFL disputes these concerns. "Any assumption that possible continuous testing will cause a shortage of supplies for the general public ignores the facts. Testing capacity has been increased massively in recent weeks," the DFL executive committee stated in a press release on Tuesday.

There are currently 640,000 tests available per week in Germany. The Bundesliga and second division would require about 0.5% of these tests.
"""
predict_from_text(text)


The predicted category using the GBC model is sport.
The conditional probability is: 79.26603669572553


In [14]:
text = """
Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2, also known as 2019-nCov or nCovid-19) outbreak has become a huge public health issue due to its rapid transmission making it a global pandemic. Currently, there are no vaccines or drugs available for nCovid-19, hence early detection is crucial to help and manage the outbreak. Here, we report an in-house built biosensor device (eCovSens) and compare it with a commercial potentiostat machine for the detection of nCovid-19 spike protein antigen (nCovid-19 Ag) in spiked saliva samples. A potentiostat based sensor was fabricated using fluorine doped tin oxide electrode (FTO) drop casted with gold nanoparticle (AuNPs) and immobilized with nCovid-19 monoclonal antibody (nCovid-19 Ab) to measure change in the electrical conductivity. Similarly, eCovSens was used to measure change in electrical conductivity by immobilizing nCovid-19 Ab on screen printed carbon electrode (SPCE). The performances of both sensors were recorded upon interaction of nCovid-19 Ab with its specific nCovid-19 Ag. Under optimum conditions, the FTO based immunosensor and proposed SPCE-based biosensor device displayed high sensitivity for early detection of nCovid-19 Ag, ranging from 1 fM to 1 uM. Our in-house developed eCovSens device can successfully detect nCovid-19 Ag at 10 fM concentration in standard buffer that is in close agreement with FTO/AuNPs sensor where AuNPs were used for the amplification of the electrical signal. The limit of detection (LOD) was found to be 90 fM with eCovSens and 120 fM with potentiostst in case of spiked saliva samples. The proposed portable point of care (PoC) eCovSens device can be used as an alternative diagnostic tool for the rapid (within 10-30 s) detection of nCovid-19 Ag traces directly in patient saliva samples that displayed high sensitivity, stability, and specificity.

"""
predict_from_text(text)

The predicted category using the GBC model is ncovid.
The conditional probability is: 88.00196234640686


In [15]:
text= """
The time of survival and the conditions affecting the 2019-nCoV viability in the environment are currently unknown. According to studies assessing the environmental stability of other coronaviruses, the Severe Acute Respiratory Syndrome coronavirus (SARS-CoV) is estimated to survive several days in the environment and the Middle East Respiratory Syndrome-related coronavirus (MERS-CoV) more than 48 hours at an average room temperature (20°C) on different surfaces. That room temperature is that of Europe or North America. Room temperature in sub-Sahara Africa is comparatively higher. It remains to be seen how the virus thrives under some very harsh climatic conditions.
"""

predict_from_text(text)

The predicted category using the GBC model is ncovid.
The conditional probability is: 99.99272756430582


In [16]:
text = """
Preprint High Temperature and High Humidity Reduce the Transmission of COVID-19 March 2020 Jingyuan WangJingyuan WangKe TangKai FengWeifeng Lv
This paper investigates how air temperature and humidity influence the transmission of COVID-19. After estimating the serial interval of COVID-19 from 105 pairs of the virus carrier and the infected, we calculate the daily effective reproductive number, R, for each of all 100 Chinese cities with more than 40 cases. Using the average R from January 21 to 23, 2020 as a proxy of non-intervened transmission intensity, we find, under a linear regression framework for 100 Chinese cities, high temperature and high relative humidity reduce the transmission of COVID-19 with a significance level of 1% and 5%, respectively, even after controlling for population density and GDP per capita of cities. One degree Celsius increase in temperature and one percent increase in relative humidity lower R by 0.0266 and 0.0106, respectively. This result is consistent with the fact that the high temperature and high humidity significantly reduce the transmission of influenza. It indicates that the arrival of summer and rainy season in the northern hemisphere can effectively reduce the transmission of the COVID-19.
"""
predict_from_text(text)

The predicted category using the GBC model is ncovid.
The conditional probability is: 99.80331582840871
